# Project IV: [OII] fluxes <a class="tocSkip">
    
the aim of this notebook is to combine the HII-region and cluster catalogues. 
    
In this notebook we do the matching on a per galaxie basis. For each resolution, a set of output files is produced that matches the nebulae to the association catalogue


In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

from astrotools.packages import *

from astrotools.constants import tab10, single_column, two_column

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
logging.basicConfig(stream=sys.stdout,datefmt='%H:%M:%S',level=logging.INFO)
logger = logging.getLogger(__name__)

basedir = Path('..')  # where we save stuff (and )
data_ext = Path('a:')/'Archive' # raw data

# we use the sample table for basic galaxy properties
sample_table = ascii.read(basedir/'..'/'pnlf'/'data'/'interim'/'sample.txt')
sample_table.add_index('name')
sample_table['SkyCoord'] = SkyCoord(sample_table['R.A.'],sample_table['Dec.'])

## Read in data

the galaxies listed in `hst_sample` have a cluster catalogue. The galaxies listed in `muse_sample` have astrosat observations to measure the FUV.

In [ ]:
# choose which version of the association catalogue to use
version = 'v1p2'
HSTband = 'nuv'
scalepc = 32

name = 'NGC2835'

### MUSE (DAP + nebulae catalogues)

In [ ]:
from pnlf.auxiliary import filter_table
from pnlf.io import ReadLineMaps

p = {x:sample_table.loc[name][x] for x in sample_table.columns}

# DAP linemaps (Halpha and OIII)
filename = next((data_ext/'MUSE'/'DR2.1'/'copt'/'MUSEDAP').glob(f'{name}*.fits'))
copt_res = float(filename.stem.split('-')[1].split('asec')[0])
with fits.open(filename) as hdul:
    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                    meta=hdul['HA6562_FLUX'].header,
                    wcs=WCS(hdul['HA6562_FLUX'].header))
    Hbeta = NDData(data=hdul['HB4861_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['HB4861_FLUX_ERR'].data),
                    mask=np.isnan(hdul['HB4861_FLUX'].data),
                    meta=hdul['HB4861_FLUX'].header,
                    wcs=WCS(hdul['HB4861_FLUX'].header))
    OIII = NDData(data=hdul['OIII5006_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['OIII5006_FLUX_ERR'].data),
                    mask=np.isnan(hdul['OIII5006_FLUX'].data),
                    meta=hdul['OIII5006_FLUX'].header,
                    wcs=WCS(hdul['OIII5006_FLUX'].header)) 

# the original catalogue from Francesco
filename = data_ext/'Products'/'Nebulae_catalogs'/'Nebulae_catalogue_v2'/'Nebulae_catalogue_v2.fits'
with fits.open(filename) as hdul:
    nebulae = Table(hdul[1].data)
with fits.open(basedir/'data'/'data_v2p1'/'Nebulae_Catalogue_v2p1_OII.fits') as hdul:
    OII_fluxes = Table(hdul[1].data)
nebulae = join(nebulae,OII_fluxes,keys=['gal_name','region_ID'])

nebulae['SkyCoord'] = SkyCoord(nebulae['cen_ra']*u.deg,nebulae['cen_dec']*u.deg,frame='icrs')
nebulae.rename_columns(['cen_x','cen_y'],['x','y'])


nebulae['HIIregion'] = (nebulae['BPT_NII']==0) & (nebulae['BPT_SII']==0) & (nebulae['BPT_OI']==0)
HII_regions = filter_table(nebulae,gal_name=name,BPT_NII=0,BPT_SII=0,BPT_OI=0)
nebulae = filter_table(nebulae,gal_name=name)
nebulae.add_index('region_ID')
HII_regions.add_index('region_ID')

filename = data_ext/'Products'/'Nebulae_catalogs'/'Nebulae_catalogue_v2'/'spatial_masks'/f'{name}_nebulae_mask.fits'
with fits.open(filename) as hdul:
    nebulae_mask = NDData(hdul[0].data.astype(float),mask=Halpha.mask,meta=hdul[0].header,wcs=WCS(hdul[0].header))
    nebulae_mask.data[nebulae_mask.data==-1] = np.nan
    
print(f'{name}: {len(HII_regions)} HII-regions in final catalogue')

### HST

**white light + filter images**

In [ ]:
from cluster.io import read_associations

target  = name.lower()

# whitelight image (we set 0s to nan)
white_light_filename = data_ext / 'HST' / 'white_light' / f'{name.lower()}_white_24rgb.fits'
if white_light_filename.is_file():
    with fits.open(white_light_filename) as hdul:
        hst_whitelight = NDData(hdul[0].data,mask=hdul[0].data==0,meta=hdul[0].header,wcs=WCS(hdul[0].header))
        hst_whitelight.data[hst_whitelight.data==0] = np.nan
else:
    logging.warning('no white light image')
    
# filter image with uncertainties
filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{name.lower()}_f275w_v1_exp-drc-sci.fits'
with fits.open(filename) as hdul:
    F275 = NDData(hdul[0].data,
                  mask=hdul[0].data==0,
                  meta=hdul[0].header,
                  wcs=WCS(hdul[0].header))
filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{name.lower()}_f275w_v1_err-drc-wht.fits'
with fits.open(filename) as hdul:
    F275.uncertainty = StdDevUncertainty(hdul[0].data)
    
associations, associations_mask = read_associations(folder=data_ext/'Products'/'stellar_associations',target=target,
                                                    HSTband=HSTband,scalepc=scalepc,version=version)


print(f'{name}: {len(associations)} associations in catalogue')    
# associations mask

### SITELLE [OII]

In [ ]:
from astrotools.regions import Regions
from reproject import reproject_interp

In [ ]:
with fits.open(basedir/'data'/'data_v2p1'/'maps'/'NGC2835_OII_map_reprojected.fits') as hdul:
    OII = NDData(data=hdul[1].data,
                 meta=hdul[1].header,
                 wcs=WCS(hdul[1].header))

## Compare Strong line prescriptions

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(10,4))

for name,ax in zip(['NGC0628','NGC2835','NGC3351'],[ax1,ax2,ax3]):
    with fits.open(basedir/'data'/'data_v2p1'/'maps'/f'{name}_OII_map.fits') as hdul:
        OII_img = hdul['OII3726_FLUX'].data
        OII_header = hdul['OII3726_FLUX'].header
    
    norm = simple_norm(OII_img,clip=False,percent=98)
    ax.imshow(OII_img,norm=norm,origin='lower',cmap=plt.cm.Greys)
    ax.set_title(name)
    ax.axis('off')
    
plt.show()   

use [OII] line to calculate strong line and direct abundances

In [ ]:
from astrotools.metallicity import diagnostic_line_ratios

with fits.open(basedir / 'data' / 'data_v2p1' / 'Nebulae_Catalogue_v2p1.fits') as hdul:
    nebulae = Table(hdul[1].data)
with fits.open(basedir/'data'/'data_v2p1'/'Nebulae_Catalogue_v2p1_OII.fits') as hdul:
    OII_fluxes = Table(hdul[1].data)

nebulae_with_OII = join(nebulae,OII_fluxes,keys=['gal_name','region_ID'])
nebulae_with_OII = nebulae_with_OII[np.isin(nebulae_with_OII['gal_name'],['NGC0628','NGC2835','NGC3351','NGC4535'])]
nebulae_with_OII = nebulae_with_OII[nebulae_with_OII['OII3726_FLUX_CORR']>0]

line_ratios = diagnostic_line_ratios(nebulae_with_OII)

compare with Figure 8 in Pilyugin+2016 (looks good if R2*=1.4)

In [ ]:
from astrotools.metallicity import strong_line_metallicity_R, strong_line_metallicity_S

subsample = nebulae_with_OII[nebulae_with_OII['OII3726_FLUX_CORR']>10*nebulae_with_OII['OII3726_FLUX_CORR_ERR']].copy()
print(f'{len(subsample)} objects in sample')

# looks a lot better with 1.4*R2
subsample['OH_R'] = strong_line_metallicity_R(subsample['R2'],subsample['R3'],subsample['N2'])
subsample['OH_S'] = strong_line_metallicity_S(subsample['S2'],subsample['R3'],subsample['N2'])

fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(6,3))

ax1.plot([8.1,8.8],[8.1,8.8],color='black')
ax1.plot([8.1,8.8],[8.2,8.9],color='grey',ls='--')
ax1.plot([8.1,8.8],[8.0,8.7],color='grey',ls='--')
ax1.scatter(subsample['OH_S'],subsample['OH_R'],s=4,c=tab10[0])

ax1.set(xlim=[8.1,8.8],ylim=[8.1,8.8],
       xlabel='12+log(O/H)$_\mathrm{S}$',
       ylabel='12+log(O/H)$_\mathrm{R}$')
ax2.hist(subsample['OH_S']-subsample['OH_R'],bins=np.linspace(-0.3,0.3,20),histtype='step',color='black')
ax2.set(xlabel=r'log(O/H)$_\mathrm{S}-$log(O/H)$_\mathrm{R}$')
plt.savefig(basedir/'reports'/'12+logOH R vs S calibration.pdf',dpi=300)
plt.show()

with direct method. This requires electron temperature and density. They have to be measured in an itterative process

In [ ]:
from astrotools.metallicity import electron_density_sulfur,\
                                electron_temperature_oxygen, electron_temperature_nitrogen,\
                                electron_temperature_sulfur, oxygen_abundance_direct
   
criteria = (line_ratios['OII7319_FLUX_CORR']>7*line_ratios['OII7319_FLUX_CORR_ERR']) & (line_ratios['OII3726_FLUX_CORR']>10*line_ratios['OII3726_FLUX_CORR_ERR'])
subsample = line_ratios[criteria].copy()

subsample['OH_R'] = strong_line_metallicity_R(subsample['R2'],subsample['R3'],subsample['N2'])
subsample['OH_S'] = strong_line_metallicity_S(subsample['S2'],subsample['R3'],subsample['N2'])
    
# initial guess for the temperature
subsample['t(NII)'] = electron_temperature_nitrogen(subsample['RN2'])
subsample['t(SIII)'] = electron_temperature_sulfur(subsample['RS3'])
subsample['n(SII)']  = electron_density_sulfur(subsample['RS2'],subsample['t(NII)'])

for x in range(10):
    subsample['t(OII)'] = electron_temperature_oxygen(subsample['RO2'],subsample['n(SII)'])
    subsample['n(SII)'] = electron_density_sulfur(subsample['RS2'],subsample['t(OII)'])
    print(np.nanmean(subsample['n(SII)']))

subsample['OH_direct'] = oxygen_abundance_direct(subsample['R2'],subsample['R3'],subsample['t(OII)'],subsample['n(SII)'])


In [ ]:
fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(6,3))

ax1.plot([8.1,8.8],[8.1,8.8],color='black')
ax1.plot([8.1,8.8],[8.2,8.9],color='grey',ls='--')
ax1.plot([8.1,8.8],[8.0,8.7],color='grey',ls='--')
ax1.scatter(subsample['OH_direct'],subsample['OH_R'],s=4,c=tab10[0])
ax1.set(xlim=[8.1,8.8],ylim=[8.1,8.8],
       xlabel='12+log(O/H) direct',
       ylabel='12+log(O/H)$_\mathrm{R}$')
ax2.hist(subsample['OH_direct']-subsample['OH_R'],bins=np.linspace(-0.3,0.3,20),histtype='step',color='black')
ax2.set(xlabel=r'log(O/H) direct$-$log(O/H)$_\mathrm{R}$')
#plt.savefig('12+logOH R vs S calibration.png',dpi=600)
plt.show()

compare with Figure 7 in Perez-Montero+2017

In [ ]:
fig,ax=plt.subplots(figsize=(6,4))

ax.scatter(np.log10(subsample['R23']),subsample['OH_direct'],s=4,c=subsample['logq_D91'])
ax.set(xlim=[-0.4,1.4],ylim=[7.,9.0],
       xlabel='log R23',
       ylabel='12+log(O/H) direct')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(4,4))

ax.plot([0.5,1.5],[0.5,1.5],color='black')
ax.scatter(subsample['t(OII)'],subsample['t(NII)'],s=4,c=tab10[0])
ax.set(xlim=[0.5,1.5],ylim=[0.5,1.5],
       xlabel='t([OII])',
       ylabel='t([NII])')
#plt.savefig('12+logOH R vs S calibration.png',dpi=600)
plt.show()

## Compare fitted spectrum to line map



In [ ]:
nebulae_mask_OII = reproject_interp(nebulae_mask,
                                     output_projection=Halpha.wcs,
                                     shape_out=OII.data.shape,
                                     order='nearest-neighbor',return_footprint=False)  

In [ ]:
# the shape of the [OII] map is wrong
nebulae['OII3726_FLUX_NEW'] = np.array([np.nansum(OII.data[nebulae_mask_OII==region_ID]) for region_ID in nebulae['region_ID']])*1e20 
nebulae['HA6562_FLUX_NEW'] = np.array([np.nansum(Halpha.data[nebulae_mask.data==region_ID]) for region_ID in nebulae['region_ID']]) 

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column))

lim = [1e2,1e6]
ax.scatter(nebulae['OII3726_FLUX'],nebulae['OII3726_FLUX_NEW']/4,s=1)
ax.plot(lim,lim,color='black')
ax.set(xlabel=r'[O\,\textsc{ii}] (fitted spectrum)',ylabel=r'[O\,\textsc{ii}] (line map)',
        xscale='log',yscale='log',xlim=lim,ylim=lim)
rho,p=spearmanr(nebulae['OII3726_FLUX'],nebulae['OII3726_FLUX_NEW'],nan_policy='omit')
print(f'[OII]: rho={rho:.2f}, p-value={p:.2g}')
ax.set_title(r'$\rho=$'+f'{rho:.2f}, p-value={p:.2g}',fontsize=8)

plt.show()

## TYPHOON

In [ ]:
with fits.open(basedir/'data'/'typhoon'/'NGC2835_Halpha_WCS.fits') as hdul:
    typhoon_halpha = NDData(data=hdul[0].data,
                 meta=hdul[0].header,
                 wcs=WCS(hdul[0].header))
with fits.open(basedir/'..'/'sitelle'/'data'/'typhoon'/'N2835_TG10_sig3.50000_hiiphot_WCS.fits') as hdul:
    typhoon_mask = NDData(data=hdul[0].data,
                     meta=hdul[0].header,
                     wcs=typhoon_halpha.wcs)

names = ['HIIregion','Rgal(kpc)','Halpha','Halpha_e','Hbeta','Hbeta_e',
         'OIII5007','OIII5007_e','OIII4636','OIII4636_e',
         'NII6583','NII6583_e','SII6716,31','SII6716,31_e',
         'OII3726,29','OII3726,29_e','SII6716','SII6716_e',
         'SII6731','SII6731_e']

typhoon_catalogue = ascii.read(basedir/'data'/'typhoon'/'NGC2835_HIIdata_notdereddened_WCS.dat',names=names)
typhoon_catalogue.rename_column('HIIregion','region_ID')
with fits.open(basedir/'data'/'typhoon'/'NGC2835_typhoon_OII_fluxes.fits') as hdul:
    OII_typhoon = Table(hdul[1].data)
typhoon_catalogue = join(typhoon_catalogue,OII_typhoon,keys='region_ID')

the TYPHOON H$\alpha$ with the regions overplotted

### use the reprojected map to measure the fluxes

In [ ]:
typhoon_regions = Regions(typhoon_mask.data,projection=typhoon_mask.wcs)

typhoon_muse, footprint = reproject_interp(typhoon_mask,
                                 output_projection=nebulae_mask.wcs,
                                 shape_out=nebulae_mask.data.shape,
                                 order='nearest-neighbor',return_footprint=True)    
typhoon_regions_muse = Regions(typhoon_muse,projection=Halpha.wcs)

# show the footprint of the MUSE Halpha
_, footprint = reproject_interp(Halpha,
                                output_projection=typhoon_mask.wcs,
                                shape_out=typhoon_mask.data.shape,
                                order='nearest-neighbor',return_footprint=True) 

look at the reprojected TYPHOON masks

In [ ]:
fig = plt.figure(figsize=(1.5*two_column,1.3*two_column/3))

ax1 = fig.add_subplot(131,projection=Halpha.wcs)
norm = simple_norm(typhoon_halpha.data,clip=False,percent=99.6)
ax1.imshow(typhoon_halpha.data,norm=norm,cmap=plt.cm.gray_r)
ax1.imshow(footprint,alpha=0.4,cmap=plt.cm.Blues)
for cont in typhoon_regions.contours:
    ax1.plot(cont[:,1], cont[:,0], 'k-',lw=0.2,color='red')
ax1.set(xlabel='R.A.',ylabel='Dec.',ylim=[250,450])
ax1.set_title(r'TYPHOON H$\alpha$',fontsize=7)

ax2 = fig.add_subplot(132,projection=Halpha.wcs)
norm = simple_norm(Halpha.data,clip=False,percent=99.6)
ax2.imshow(Halpha.data,norm=norm,cmap=plt.cm.gray_r)
for cont in typhoon_regions_muse.contours:
    ax2.plot(cont[:,1], cont[:,0], 'k-',lw=0.2,color='red')
ax2.set(xlabel='R.A.',ylabel=' ')
#ax2.coords[1].set_ticklabel_visible(False)
ax2.set_title(r'MUSE H$\alpha$',fontsize=7)


ax3 = fig.add_subplot(133,projection=OII.wcs)
norm = simple_norm(OII.data,clip=False,percent=99.6)
ax3.imshow(OII.data,norm=norm,cmap=plt.cm.gray_r)
for cont in typhoon_regions_muse.contours:
    ax3.plot(cont[:,1], cont[:,0], 'k-',lw=0.2,color='red')
ax3.set(xlabel='R.A.',ylabel='Dec.')
ax3.coords[1].set_ticklabel_visible(False)
ax3.set_title('SITELLE [OII]',fontsize=7)

plt.tight_layout()
plt.savefig(basedir/'reports'/'TYPHOON_regions_reprojected.pdf',dpi=300)
plt.show()

In [ ]:
import pyneb as pn
EBV_NGC2835 = 0.089

rc_MW = pn.RedCorr(R_V=3.1,E_BV=EBV_NGC2835,law='CCM89 oD94')

columns = ['Halpha','Halpha_e','Hbeta','Hbeta_e','OIII5007','OIII5007_e','OII3726,29','OII3726,29_e']
wavelength = [6562,6562,4861,4861,5006,5006,3726,3627]

tmp = typhoon_catalogue[['region_ID']+columns].copy()
tmp = tmp[np.isin(tmp['region_ID'],typhoon_muse)]

# the TYPHOON fluxes are not extinction corrected (MW)
for col, wav in zip(columns,wavelength):
    tmp[col] *= 1e3 * rc_MW.getCorr(wav)

# the fluxes in typhon are in units of 1e-17 erg / cm2 / s
tmp['Halpha_muse'] = np.array([np.nansum(Halpha.data[typhoon_muse==region_ID]) for region_ID in tmp['region_ID']]) 
tmp['Hbeta_muse'] = np.array([np.nansum(Hbeta.data[typhoon_muse==region_ID]) for region_ID in tmp['region_ID']]) 
tmp['OIII_muse'] = np.array([np.nansum(OIII.data[typhoon_muse==region_ID]) for region_ID in tmp['region_ID']]) 
tmp['OII_sitelle'] = np.array([np.nansum(OII.data[typhoon_muse==region_ID]) for region_ID in tmp['region_ID']]) * 1e20 

# we need both lines [OII]3729 = 0.35 * [OII]3726 and extinction correction
tmp['OII_sitelle'] *= rc_MW.getCorr(3726) * 1.35

tmp['outside'] = [np.any(Halpha.mask[typhoon_muse==region_ID]) for region_ID in tmp['region_ID']]
#tmp = tmp[~tmp['outside']]
tmp['size'] = [np.sum(typhoon_muse==region_ID) for region_ID in tmp['region_ID']]
#tmp = tmp[tmp['size']>1000]

# or better use the OII fluxes from the integrated spectra
tmp = join(tmp,OII_typhoon,keys='region_ID')
tmp['OII_sitelle_fit'] = tmp['OII3726_FLUX']+tmp['OII3729_FLUX']
#rc_balmer = pn.RedCorr(R_V=3.1,law='CCM89 oD94')
#rc_balmer.setCorr(obs_over_theo=tmp['Halpha_muse']/tmp['Hbeta_muse'] / 2.86,wave1=6562.81,wave2=4861.33)

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column))

lim = [1e5,5e7]
ax.scatter(tmp['OII_sitelle_fit'],tmp['OII_sitelle'],s=1)
ax.plot(lim,lim,color='black')
ax.set(xlabel=r'[O\,\textsc{ii}] (integrated spectra)',ylabel=r'[O\,\textsc{ii}] (map)',
        xscale='log',yscale='log',xlim=lim,ylim=lim)
rho,p=spearmanr(tmp['OII3726,29'],tmp['OII_sitelle'],nan_policy='omit')
print(f'[OII]: rho={rho:.2f}, p-value={p:.2g}')
ax.set_title(r'$\rho=$'+f'{rho:.2f}, p-value={p:.2g}',fontsize=8)
plt.savefig(basedir/'reports'/'integrated_spectra_vs_map.png',dpi=300)

plt.show()

In [ ]:
from scipy.stats import spearmanr

fig,((ax1,ax2,ax3),(ax4,ax5,ax6))=plt.subplots(ncols=3,nrows=2,figsize=(two_column,two_column/1.5))
factor = 1
kwargs = {'c' : tmp['size'], 'vmin':0,'vmax':4000}
kwargs = {}

lim = [1e5,2e7]
ax1.scatter(tmp['Halpha'],tmp['Halpha_muse'],s=1,**kwargs)
ax1.plot(lim,lim,color='black')
ax1.set(xlabel=r'H$\alpha$ (TYPHOON)',ylabel=r'H$\alpha$ (MUSE)',
        xscale='log',yscale='log',xlim=lim,ylim=lim)
rho,p=spearmanr(tmp['Halpha'],tmp['Halpha_muse'],nan_policy='omit')
print(f'Halpha: rho={rho:.2f}, p-value={p:.2g}')
#ax1.set_title(r'$\rho=$'+f'{rho:.2f}, p-value={p:.2g}',fontsize=8)

lim = [1e5,2e7]
ax2.scatter(tmp['Hbeta'],tmp['Hbeta_muse'],s=1,**kwargs)
ax2.plot(lim,lim,color='black')
ax2.set(xlabel=r'H$\beta$ (TYPHOON)',ylabel=r'H$\beta$ (MUSE)',
        xscale='log',yscale='log',xlim=lim,ylim=lim)
rho,p=spearmanr(tmp['Hbeta'],tmp['Hbeta_muse'],nan_policy='omit')
print(f'Hbeta: rho={rho:.2f}, p-value={p:.2g}')
#ax2.set_title(r'$\rho=$'+f'{rho:.2f}, p-value={p:.2g}',fontsize=8)

lim = [1e5,5e7]
ax3.scatter(tmp['OII3726,29'],tmp['OII_sitelle_fit']/factor,s=1,**kwargs)
ax3.plot(lim,lim,color='black')
ax3.set(xlabel=r'[O\,\textsc{ii}] (TYPHOON)',ylabel=r'[O\,\textsc{ii}] (SITELLE)',
        xscale='log',yscale='log',xlim=lim,ylim=lim)
rho,p=spearmanr(tmp['OII3726,29'],tmp['OII_sitelle_fit'],nan_policy='omit')
print(f'[OII]: rho={rho:.2f}, p-value={p:.2g}')
#ax3.set_title(r'$\rho=$'+f'{rho:.2f}, p-value={p:.2g}',fontsize=8)

lim = [0,1]
ax4.scatter(tmp['OIII5007']/tmp['Halpha'],tmp['OIII_muse']/tmp['Halpha_muse'],s=1,**kwargs)
ax4.plot(lim,lim,color='black')
ax4.set(xlim=lim,ylim=lim,xlabel=r'[O\,\textsc{iii}]/H$\alpha$ (TYPHOON)',ylabel=r'[O\,\textsc{iii}]/H$\alpha$ (MUSE)')
rho,p=spearmanr(tmp['OIII5007']/tmp['Halpha'],tmp['OIII_muse']/tmp['Halpha_muse'],nan_policy='omit')
print(f'OIII/Halpha: rho={rho:.2f}, p-value={p:.2g}')
#ax4.set_title(r'$\rho=$'+f'{rho:.2f}, p-value={p:.2g}',fontsize=8)

lim=[0,2]
ax5.scatter(tmp['OII3726,29']/tmp['Halpha'],tmp['OII_sitelle_fit']/tmp['Halpha_muse']/factor,s=1,**kwargs)
ax5.plot(lim,lim,color='black')
ax5.set(xlim=lim,ylim=lim,xlabel=r'[O\,\textsc{ii}]/H$\alpha$ (TYPHOON)',ylabel=r'[O\,\textsc{ii}]/H$\alpha$ (SITELLE/MUSE)')
rho,p=spearmanr(tmp['OII3726,29']/tmp['Halpha'],tmp['OII_sitelle_fit']/tmp['Halpha_muse'],nan_policy='omit')
print(f'OII/Halpha: rho={rho:.2f}, p-value={p:.2g}')
#ax5.set_title(r'$\rho=$'+f'{rho:.2f}, p-value={p:.2g}',fontsize=8)

lim=[0,4.5]
ax6.scatter(tmp['OII3726,29']/tmp['Hbeta'],tmp['OII_sitelle_fit']/tmp['Hbeta_muse']/factor,s=1,**kwargs)
ax6.plot(lim,lim,color='black')
ax6.set(xlim=lim,ylim=lim,xlabel=r'[O\,\textsc{ii}]/H$\beta$ (TYPHOON)',ylabel=r'[O\,\textsc{ii}]/H$\beta$ (SITELLE/MUSE)')
rho,p=spearmanr(tmp['OII3726,29']/tmp['Hbeta'],tmp['OII_sitelle_fit']/tmp['Hbeta_muse'],nan_policy='omit')
print(f'OII/Hbeta: rho={rho:.2f}, p-value={p:.2g}')
#ax6.set_title(r'$\rho=$'+f'{rho:.2f}, p-value={p:.2g}',fontsize=8)

plt.tight_layout()
plt.savefig(basedir/'reports'/'TYPHOON_flux_comparison.pdf',dpi=300)
plt.show()

### use the reprojected regions to measure the flux

In [ ]:
typhoon_regions = Regions(typhoon_mask.data,projection=typhoon_mask.wcs)
typhoon_regions_muse = typhoon_regions.reproject(Halpha.wcs,shape=Halpha.data.shape)